<a href="https://colab.research.google.com/github/pkams/coursera_TensorFlowSpecialization/blob/master/Modulo3_3_LSTM_GRU_Conv1D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=87c8c4742b09a1eb2b4d59712fa87635307092c78557d7c3d0cd634a9c0b4da3
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [2]:
import json
import tensorflow as tf
import csv
import random
import numpy as np
import pandas as pd
import wget

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers

In [3]:
# Declarando as váriaveis gerais

embedding_dim = 100 #(dimensão do embedding)
max_length = 100
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'
train_portion = .9


In [4]:
url = 'https://storage.googleapis.com/laurencemoroney-blog.appspot.com/training_cleaned.csv'
wget.download(url)

'training_cleaned.csv'

In [5]:
df = pd.read_csv('training_cleaned.csv', header=None)
df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [6]:
corpus = df.loc[:, [0,5]]

In [7]:
corpus.loc[:,0] = corpus.loc[:,0].apply(lambda x: 0 if x == 0 else 1)
corpus.loc[:,0]

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: 0, Length: 1600000, dtype: int64

In [8]:
# Shuffle no dataframe
corpus = corpus.sample(frac=1).reset_index(drop=True)
corpus

,0,5
0,1,@MeganMac awwww thanks Megan!! I missed you to...
1,1,@tommcfly cant beat a bit of cheesiness thoug...
2,0,@venomcandy wow... she gave in O.O so did I
3,0,today i received some very devastating news. D...
4,1,The cows are lying down... RAINYDAYTIMES
...,...,...
1599995,1,on my way to the bank
1599996,1,woo hoo i hope they play the hokey pokey sammi...
1599997,1,watching cbbc. im a little old 4 that but im b...
1599998,0,First day of summer vacation tom...but I have ...


In [9]:
train_size = int(len(corpus)*train_portion)
train_size

1440000

In [10]:
sentences_train = corpus[5][:train_size].values
sentences_test = corpus[5][train_size:].values
labels_train = corpus[0][:train_size].values
labels_test = corpus[0][train_size:].values

print(sentences_train.shape) 
print(sentences_test.shape)
print(labels_train.shape) 
print(labels_test.shape)

(1440000,)
(160000,)
(1440000,)
(160000,)


In [11]:
# Fazendo tokenizadores
tokenizer = Tokenizer()
# Fitando no treino
tokenizer.fit_on_texts(sentences_train)
# Salvando word index
word_index = tokenizer.word_index
# Transformando em sequencia
train_sequence = tokenizer.texts_to_sequences(sentences_train)
# Padding
train_pad = pad_sequences(train_sequence, padding=padding_type, maxlen=max_length)

In [12]:
# Aplicando no treino
test_sequence = tokenizer.texts_to_sequences(sentences_test)
# Padding
test_pad = pad_sequences(test_sequence, padding=padding_type, maxlen=max_length)

In [13]:
# Ver tamanho do vocab criado
vocab_size = len(word_index)
print(vocab_size)

643908


In [14]:
# Baixando embedding matrix pronta (com pesos já definidos)

url = 'https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt'
wget.download(url)

# Loading dos pesos e coeficientes
embeddings_index = {};
with open('glove.6B.100d.txt') as f:
    for line in f:
        values = line.split();
        word = values[0];
        coefs = np.asarray(values[1:], dtype='float32');
        embeddings_index[word] = coefs;

embeddings_matrix = np.zeros((vocab_size+1, embedding_dim));
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word);
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector;

In [15]:
len(embeddings_matrix)

643909

In [18]:
# Criando modelo

model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=max_length, weights=[embeddings_matrix], trainable=False),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Conv1D(64, 5, activation='relu'),
        tf.keras.layers.MaxPooling1D(pool_size=4),
        tf.keras.layers.LSTM(64),
        tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          64390900  
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 100)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 96, 64)            32064     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 24, 64)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 64,456,053
Trainable params: 65,153
Non-trainable params: 64,390,900
_____________________________________

In [20]:
model.fit(train_pad, 
          labels_train,
          epochs=10,
          validation_data=(test_pad, labels_test),
          verbose=1)

Epoch 1/10
45000/45000 [==============================] - 199s 4ms/step - loss: 0.4875 - accuracy: 0.7623 - val_loss: 0.4550 - val_accuracy: 0.7839
Epoch 2/10
45000/45000 [==============================] - 194s 4ms/step - loss: 0.4643 - accuracy: 0.7775 - val_loss: 0.4454 - val_accuracy: 0.7907
Epoch 3/10
45000/45000 [==============================] - 196s 4ms/step - loss: 0.4570 - accuracy: 0.7821 - val_loss: 0.4422 - val_accuracy: 0.7922
Epoch 4/10
45000/45000 [==============================] - 194s 4ms/step - loss: 0.4528 - accuracy: 0.7849 - val_loss: 0.4404 - val_accuracy: 0.7950
Epoch 5/10
45000/45000 [==============================] - 193s 4ms/step - loss: 0.4500 - accuracy: 0.7864 - val_loss: 0.4364 - val_accuracy: 0.7947
Epoch 6/10
45000/45000 [==============================] - 193s 4ms/step - loss: 0.4483 - accuracy: 0.7875 - val_loss: 0.4368 - val_accuracy: 0.7955
Epoch 7/10
45000/45000 [==============================] - 193s 4ms/step - loss: 0.4473 - accuracy: 0.7880 - val_